In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the train data
train_df = pd.read_csv('Train.csv')

# Define image paths and labels
image_dir = 'train'  # Directory containing training images
train_df['./images'] = train_df['ID'].apply(lambda x: os.path.join(image_dir, x + '.jpg'))

# Load images and resize them
def load_image(img_path, img_size):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (img_size, img_size))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0  # Normalize
    return img

img_size = 224  # Resize images to 224x224 for EfficientNet
X = np.array([load_image(img_path, img_size) for img_path in train_df['image_path']])
y = pd.get_dummies(train_df['Damage']).values  # One-hot encode the labels

# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator()

# Pretrained model (EfficientNet)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(y_train.shape[1], activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for early stopping and model checkpointing
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', save_best_only=True)
]

# Train the model
model.fit(
    train_datagen.flow(X_train, y_train, batch_size=32),
    validation_data=val_datagen.flow(X_val, y_val),
    epochs=20,
    callbacks=callbacks
)

# Load the test data
test_df = pd.read_csv('Test.csv')
test_df['image_path'] = test_df['ID'].apply(lambda x: os.path.join('test', x + '.jpg'))
X_test = np.array([load_image(img_path, img_size) for img_path in test_df['image_path']])

# Predict on the test set
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)

# Map predicted labels back to the original labels
label_mapping = {i: label for i, label in enumerate(train_df['Damage'].unique())}
test_df['Damage'] = predicted_labels.map(label_mapping)

# Prepare the submission file
submission_df = test_df[['ID', 'Damage']]
submission_df.to_csv('submission.csv', index=False)
